In [ ]:
# Klasifikasi Koin Rupiah - Versi Diperbaiki & Optimized

import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from skimage.feature import hog

# =============================================
# 1. EKSTRAKSI FITUR (KONSISTEN)
# =============================================
def extract_features(img):
    """
    Ekstraksi fitur dari gambar koin dengan fitur tambahan untuk membedakan nominal
    Returns: features vector, gray image, threshold image
    """
    # Resize dulu untuk konsistensi
    img_resized = cv2.resize(img, (128, 128))
    
    # Grayscale
    if len(img_resized.shape) == 3:
        gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    else:
        gray = img_resized
    
    # Preprocessing dengan CLAHE untuk meningkatkan kontras
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray_clahe = clahe.apply(gray)
    
    # Otsu threshold
    _, th = cv2.threshold(
        gray_clahe, 0, 255,
        cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )
    
    # HOG features dari threshold image
    hog_feat = hog(
        th,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )
    
    # HOG features dari grayscale (lebih detail)
    hog_feat_gray = hog(
        gray_clahe,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )
    
    # Hu Moments (Shape descriptor)
    moments = cv2.moments(th)
    hu_moments = cv2.HuMoments(moments)
    hu_feat = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
    hu_feat = hu_feat.flatten()
    
    # Global statistics
    white_ratio = np.sum(th == 255) / th.size
    black_ratio = np.sum(th == 0) / th.size
    mean_intensity = np.mean(gray_clahe)
    std_intensity = np.std(gray_clahe)
    
    # Texture features menggunakan histogram
    hist = cv2.calcHist([gray_clahe], [0], None, [16], [0, 256])
    hist = hist.flatten() / hist.sum()  # Normalize
    
    # Central patch (area angka) - PENTING untuk membedakan nominal
    h, w = gray_clahe.shape
    
    # Patch tengah (area angka)
    center_patch = gray_clahe[int(h*0.4):int(h*0.75), int(w*0.3):int(w*0.7)]
    
    if center_patch.size > 0:
        edges_center = cv2.Canny(center_patch, 50, 150)
        edge_density_center = np.sum(edges_center > 0) / edges_center.size
        center_mean = np.mean(center_patch)
        center_std = np.std(center_patch)
    else:
        edge_density_center = 0
        center_mean = 0
        center_std = 0
    
    # Bottom patch (untuk nominal di bawah)
    bottom_patch = gray_clahe[int(h*0.6):int(h*0.9), int(w*0.25):int(w*0.75)]
    
    if bottom_patch.size > 0:
        edges_bottom = cv2.Canny(bottom_patch, 50, 150)
        edge_density_bottom = np.sum(edges_bottom > 0) / edges_bottom.size
    else:
        edge_density_bottom = 0
    
    # Radius analysis (untuk membedakan ukuran koin)
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        perimeter = cv2.arcLength(largest_contour, True)
        
        if perimeter > 0:
            circularity = 4 * np.pi * area / (perimeter * perimeter)
        else:
            circularity = 0
    else:
        area = 0
        perimeter = 0
        circularity = 0
    
    # Gabungkan semua fitur
    features = np.hstack([
        hog_feat,
        hog_feat_gray,
        hu_feat,
        hist,
        white_ratio,
        black_ratio,
        mean_intensity,
        std_intensity,
        edge_density_center,
        edge_density_bottom,
        center_mean,
        center_std,
        area,
        perimeter,
        circularity
    ])
    
    return features, gray, th


# =============================================
# 2. LOAD DATASET
# =============================================
def load_dataset(dataset_path, use_side=False):
    """
    Load dataset koin
    
    Args:
        dataset_path: Path ke folder dataset
        use_side: True = klasifikasi depan/belakang, False = hanya nominal
    
    Returns:
        X: feature array
        y: label array
        label_map: mapping label ke nama kelas
    """
    X = []
    y = []
    
    # Definisi kelas
    CLASSES = ["100_depan", "100_belakang",
               "200_depan", "200_belakang",
               "500_depan", "500_belakang",
               "1000_depan", "1000_belakang"]
    
    for label in CLASSES:
        folder = os.path.join(dataset_path, label)
        if not os.path.isdir(folder):
            print(f"Warning: Folder {folder} tidak ditemukan")
            continue
        
        files = os.listdir(folder)
        print(f"Loading {label}: {len(files)} images")
        
        for file in files:
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path)
            
            if img is None:
                print(f"Warning: Gagal membaca {img_path}")
                continue
            
            try:
                features, _, _ = extract_features(img)
                X.append(features)
                
                # Jika use_side=False, ambil hanya nominal
                if not use_side:
                    y.append(label.split('_')[0])  # "100_depan" -> "100"
                else:
                    y.append(label)
                    
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
                continue
    
    X = np.array(X)
    y = np.array(y)
    
    # Buat label mapping
    unique_labels = sorted(list(set(y)))
    label_map = {label: i for i, label in enumerate(unique_labels)}
    
    print(f"\nTotal data loaded: {len(X)}")
    print(f"Classes: {unique_labels}")
    print(f"Class distribution:")
    for label in unique_labels:
        count = np.sum(y == label)
        print(f"  {label}: {count} samples")
    
    return X, y, label_map


# =============================================
# 3. TRAINING MODEL (OPTIMIZED)
# =============================================
def train_model(X_train, y_train, X_val=None, y_val=None, fast_mode=True):
    """
    Training SVM classifier dengan GridSearch
    
    Args:
        fast_mode: True = parameter lebih sedikit (cepat), False = extensive search (akurat)
    """
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("svm", SVC(random_state=42, class_weight='balanced'))
    ])
    
    if fast_mode:
        # FAST MODE: Parameter lebih sedikit (< 2 menit)
        param_grid = {
            "svm__C": [1, 10, 100],
            "svm__gamma": ["scale", 0.01],
            "svm__kernel": ["rbf"]  # Hanya RBF (paling umum untuk koin)
        }
        cv_folds = 3
    else:
        # EXTENSIVE MODE: Parameter lengkap (8+ menit)
        param_grid = {
            "svm__C": [0.1, 1, 10, 100, 1000],
            "svm__gamma": ["scale", "auto", 0.1, 0.01, 0.001],
            "svm__kernel": ["rbf", "poly", "linear"]
        }
        cv_folds = 5
    
    total_combinations = len(param_grid['svm__C']) * len(param_grid['svm__gamma']) * len(param_grid['svm__kernel'])
    
    print("\nTraining model dengan GridSearchCV...")
    print(f"Mode: {'FAST' if fast_mode else 'EXTENSIVE'}")
    print(f"Total kombinasi parameter: {total_combinations}")
    print(f"CV folds: {cv_folds}")
    print(f"Estimasi waktu: {'< 2 menit' if fast_mode else '8+ menit'}")
    
    grid = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv_folds,
        scoring="accuracy",
        n_jobs=-1,
        verbose=1
    )
    
    import time
    start_time = time.time()
    grid.fit(X_train, y_train)
    elapsed_time = time.time() - start_time
    
    print(f"\n✅ Training selesai dalam {elapsed_time:.1f} detik ({elapsed_time/60:.1f} menit)")
    print(f"Best parameters: {grid.best_params_}")
    print(f"Best CV score: {grid.best_score_:.3f}")
    
    # Evaluasi di validation set jika ada
    if X_val is not None and y_val is not None:
        val_score = grid.score(X_val, y_val)
        print(f"Validation accuracy: {val_score:.3f}")
    
    return grid.best_estimator_


# =============================================
# 4. EVALUASI MODEL
# =============================================
def evaluate_model(model, X_test, y_test):
    """
    Evaluasi model dan tampilkan metrics
    """
    y_pred = model.predict(X_test)
    
    print("\n" + "="*50)
    print("CLASSIFICATION REPORT")
    print("="*50)
    print(classification_report(y_test, y_pred))
    
    print("\n" + "="*50)
    print("CONFUSION MATRIX")
    print("="*50)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    # Visualisasi confusion matrix
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    classes = sorted(list(set(y_test)))
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    # Tambahkan angka di setiap cell
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()
    
    return y_pred


# =============================================
# 5. DETEKSI DAN PREDIKSI MULTIPLE COINS
# =============================================
def detect_and_predict(image, model, show_confidence=True):
    """
    Deteksi dan klasifikasi multiple coins dalam satu gambar
    """
    output = image.copy()
    results = []
    
    # Preprocessing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 2)
    
    # Otsu threshold
    _, th = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Invert jika background terang
    if np.mean(th) > 127:
        th = cv2.bitwise_not(th)
    
    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    th = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel)
    th = cv2.morphologyEx(th, cv2.MORPH_OPEN, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return output, []
    
    # Process each contour
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        
        if area < 3000:
            continue
        
        # Get bounding circle
        (x, y), r = cv2.minEnclosingCircle(c)
        x, y, r = int(x), int(y), int(r)
        
        # Extract ROI dengan padding
        padding = 10
        y1 = max(0, y - r - padding)
        y2 = min(image.shape[0], y + r + padding)
        x1 = max(0, x - r - padding)
        x2 = min(image.shape[1], x + r + padding)
        
        roi = image[y1:y2, x1:x2]
        
        if roi.size == 0 or roi.shape[0] < 50 or roi.shape[1] < 50:
            continue
        
        try:
            # Extract features dan prediksi
            features, gray_roi, th_roi = extract_features(roi)
            pred = model.predict([features])[0]
            
            # Get confidence scores
            if hasattr(model.named_steps['svm'], 'decision_function'):
                decision_scores = model.decision_function([features])[0]
                confidence_scores = np.exp(decision_scores) / np.sum(np.exp(decision_scores))
                classes = model.classes_
                
                pred_idx = np.where(classes == pred)[0][0]
                confidence = confidence_scores[pred_idx]
            else:
                confidence = 1.0
                confidence_scores = None
                classes = None
            
            # Visualisasi
            color = (0, 255, 0)  # Green
            
            if show_confidence and confidence < 0.5:
                color = (0, 165, 255)  # Orange untuk low confidence
            
            cv2.circle(output, (x, y), r, color, 3)
            
            # Label dengan background
            if show_confidence:
                label = f"Rp {pred} ({confidence*100:.0f}%)"
            else:
                label = f"Rp {pred}"
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1.0
            thickness = 2
            
            (text_width, text_height), baseline = cv2.getTextSize(label, font, font_scale, thickness)
            
            cv2.rectangle(
                output,
                (x - r, y - r - text_height - baseline - 10),
                (x - r + text_width, y - r),
                (0, 0, 255),
                -1
            )
            
            cv2.putText(
                output,
                label,
                (x - r, y - r - 10),
                font,
                font_scale,
                (255, 255, 255),
                thickness
            )
            
            results.append((gray_roi, th_roi, pred, confidence_scores, classes))
            
        except Exception as e:
            print(f"Error processing coin {i}: {e}")
            continue
    
    return output, results


# =============================================
# 5B. DEBUGGING FUNCTION
# =============================================
def analyze_misclassification(model, X_test, y_test):
    """
    Analisis kesalahan klasifikasi untuk debugging
    """
    y_pred = model.predict(X_test)
    
    print("\n" + "="*60)
    print("ANALISIS KESALAHAN KLASIFIKASI")
    print("="*60)
    
    misclass_indices = np.where(y_test != y_pred)[0]
    
    if len(misclass_indices) == 0:
        print("✅ Tidak ada kesalahan klasifikasi! Perfect accuracy!")
        return
    
    print(f"\nTotal kesalahan: {len(misclass_indices)} dari {len(y_test)} ({len(misclass_indices)/len(y_test)*100:.1f}%)")
    print("\nDetail kesalahan:")
    print("-" * 60)
    
    for idx in misclass_indices:
        true_label = y_test[idx]
        pred_label = y_pred[idx]
        
        decision_scores = model.decision_function([X_test[idx]])[0]
        confidence_scores = np.exp(decision_scores) / np.sum(np.exp(decision_scores))
        classes = model.classes_
        
        print(f"\nSample {idx}:")
        print(f"  True label:      Rp {true_label}")
        print(f"  Predicted label: Rp {pred_label}")
        print(f"  Confidence scores:")
        
        sorted_indices = np.argsort(confidence_scores)[::-1]
        for i in sorted_indices:
            print(f"    Rp {classes[i]:4s}: {confidence_scores[i]*100:5.1f}%")
    
    print("\n" + "="*60)
    print("PASANGAN YANG SERING SALAH")
    print("="*60)
    
    confusion_pairs = {}
    for idx in misclass_indices:
        pair = (y_test[idx], y_pred[idx])
        confusion_pairs[pair] = confusion_pairs.get(pair, 0) + 1
    
    sorted_pairs = sorted(confusion_pairs.items(), key=lambda x: x[1], reverse=True)
    for (true, pred), count in sorted_pairs:
        print(f"Rp {true} → Rp {pred}: {count}x")


# =============================================
# 5C. AUGMENTATION FUNCTION
# =============================================
def augment_data(X, y, augmentation_factor=3):
    """
    Augmentasi data dengan transformasi sederhana
    """
    print(f"\nMelakukan augmentasi data (factor={augmentation_factor})...")
    
    X_aug = [X]
    y_aug = [y]
    
    for i in range(augmentation_factor - 1):
        noise = np.random.normal(0, 0.01, X.shape)
        X_noisy = X + noise
        
        X_aug.append(X_noisy)
        y_aug.append(y)
    
    X_final = np.vstack(X_aug)
    y_final = np.hstack(y_aug)
    
    print(f"Data sebelum augmentasi: {len(X)}")
    print(f"Data setelah augmentasi: {len(X_final)}")
    
    return X_final, y_final


# =============================================
# 5D. SAVE/LOAD MODEL
# =============================================
def save_model(model, filepath="coin_classifier_model.pkl"):
    """Save trained model ke file"""
    import pickle
    
    with open(filepath, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"\n✅ Model berhasil disimpan ke: {filepath}")


def load_model(filepath="coin_classifier_model.pkl"):
    """Load trained model dari file"""
    import pickle
    
    if not os.path.exists(filepath):
        return None
    
    with open(filepath, 'rb') as f:
        model = pickle.load(f)
    
    print(f"\n✅ Model berhasil dimuat dari: {filepath}")
    return model


# =============================================
# 6. GRADIO INTERFACE
# =============================================
def create_gradio_interface(model):
    """Membuat Gradio interface untuk testing model"""
    import gradio as gr
    
    def process_image(image, show_confidence):
        if image is None:
            return None, [], "Tidak ada gambar yang diupload"
        
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        output, results = detect_and_predict(image_bgr, model, show_confidence=show_confidence)
        output_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
        
        gallery_images = []
        predictions = []
        
        for i, result in enumerate(results):
            gray_roi, th_roi, pred = result[0], result[1], result[2]
            
            gray_rgb = cv2.cvtColor(gray_roi, cv2.COLOR_GRAY2RGB)
            th_rgb = cv2.cvtColor(th_roi, cv2.COLOR_GRAY2RGB)
            
            gallery_images.append((gray_rgb, f"Coin {i+1}: Grayscale"))
            gallery_images.append((th_rgb, f"Coin {i+1}: Threshold"))
            
            predictions.append(pred)
        
        if len(results) == 0:
            summary = "❌ Tidak ada koin terdeteksi\n\n"
            summary += "Tips:\n"
            summary += "- Pastikan pencahayaan cukup\n"
            summary += "- Koin tidak terlalu berdekatan\n"
            summary += "- Background kontras dengan koin"
        else:
            summary = f"✅ Terdeteksi {len(results)} koin\n\n"
            summary += "Hasil Klasifikasi:\n"
            summary += "-" * 40 + "\n"
            
            for i, result in enumerate(results):
                pred = result[2]
                confidence_scores = result[3]
                classes = result[4]
                
                if confidence_scores is not None and classes is not None:
                    sorted_indices = np.argsort(confidence_scores)[::-1][:3]
                    
                    summary += f"\nKoin {i+1}: Rp {pred}\n"
                    summary += "  Top 3 predictions:\n"
                    for idx in sorted_indices:
                        summary += f"    Rp {classes[idx]:4s}: {confidence_scores[idx]*100:5.1f}%\n"
                else:
                    summary += f"Koin {i+1}: Rp {pred}\n"
            
            summary += "\n" + "=" * 40 + "\n"
            
            from collections import Counter
            coin_counts = Counter(predictions)
            
            summary += "\nRingkasan:\n"
            total_value = 0
            for coin, count in sorted(coin_counts.items()):
                coin_value = int(coin)
                subtotal = coin_value * count
                total_value += subtotal
                summary += f"- Rp {coin}: {count}x = Rp {subtotal:,}\n"
            
            summary += f"\n💰 TOTAL NILAI: Rp {total_value:,}"
        
        return output_rgb, gallery_images, summary
    
    with gr.Blocks(title="Klasifikasi Koin Rupiah", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🪙 Klasifikasi Koin Rupiah
            
            Upload gambar koin Rupiah untuk deteksi dan klasifikasi otomatis.
            
            **Denominasi yang didukung:** Rp 100, Rp 200, Rp 500, Rp 1000
            
            **Metode:** Otsu Thresholding + HOG + Hu Moments + SVM Classifier
            """
        )
        
        with gr.Row():
            with gr.Column(scale=1):
                input_image = gr.Image(label="📷 Upload Gambar Koin", type="numpy")
                
                show_confidence = gr.Checkbox(
                    label="Tampilkan Confidence Scores",
                    value=True,
                    info="Menampilkan tingkat kepercayaan prediksi"
                )
                
                with gr.Row():
                    clear_btn = gr.ClearButton(components=[input_image], value="🗑️ Clear")
                    submit_btn = gr.Button("🔍 Detect & Classify", variant="primary")
                
                summary_output = gr.Textbox(label="📊 Ringkasan Hasil", lines=15, interactive=False)
            
            with gr.Column(scale=1):
                output_image = gr.Image(label="✨ Hasil Deteksi & Klasifikasi")
        
        with gr.Row():
            gallery_output = gr.Gallery(
                label="🔬 Preprocessing Steps (Grayscale & Threshold)",
                columns=4,
                height="auto"
            )
        
        submit_btn.click(
            fn=process_image,
            inputs=[input_image, show_confidence],
            outputs=[output_image, gallery_output, summary_output]
        )
        
        gr.Markdown(
            """
            ---
            ### 💡 Tips untuk Hasil Terbaik:
            
            1. **Pencahayaan**: Gunakan pencahayaan yang cukup dan merata
            2. **Background**: Gunakan background yang kontras (gelap untuk koin terang)
            3. **Jarak**: Hindari koin yang terlalu berdekatan atau bertumpuk
            4. **Fokus**: Pastikan gambar tidak blur
            5. **Sudut**: Foto dari atas tegak lurus dengan koin
            
            ### 🔍 Interpretasi Confidence Scores:
            - **> 80%**: Prediksi sangat yakin ✅
            - **60-80%**: Prediksi cukup yakin ⚠️
            - **< 60%**: Prediksi kurang yakin, mungkin perlu foto lebih baik ❌
            
            ### 🛠️ Teknologi:
            - **Preprocessing**: CLAHE, Otsu Thresholding, Morphological Operations
            - **Feature Extraction**: 
              - HOG (Histogram of Oriented Gradients)
              - Hu Moments (Shape descriptor)
              - Texture Histogram
              - Edge Density Analysis
              - Circularity & Geometry features
            - **Classifier**: Support Vector Machine (SVM) with RBF kernel
            """
        )
    
    return demo


# =============================================
# MAIN EXECUTION
# =============================================
if __name__ == "__main__":
    import sys
    import time
    
    overall_start = time.time()
    
    # KONFIGURASI
    DATASET_PATH = "dataset2"
    MODEL_PATH = "coin_classifier_model.pkl"
    
    # Set ini ke True jika ingin SKIP training dan langsung pakai model tersimpan
    USE_SAVED_MODEL = False
    
    # Training settings
    USE_AUGMENTATION = True
    AUGMENTATION_FACTOR = 2
    FAST_MODE = True  # True = cepat (~2 menit), False = akurat (~8 menit)
    
    # ==========================================
    # 1. LOAD ATAU TRAIN MODEL
    # ==========================================
    
    if USE_SAVED_MODEL:
        print("="*60)
        print("LOADING SAVED MODEL")
        print("="*60)
        model = load_model(MODEL_PATH)
        
        if model is None:
            print("❌ Model tidak ditemukan! Training model baru...")
            USE_SAVED_MODEL = False
        else:
            print("✅ Menggunakan model tersimpan. Skip training.")
    
    if not USE_SAVED_MODEL:
        # Load dataset
        print("="*60)
        print("LOADING DATASET")
        print("="*60)
        X, y, label_map = load_dataset(DATASET_PATH, use_side=False)
        
        if len(X) == 0:
            print("\n❌ Error: Dataset kosong!")
            print("Pastikan path dataset benar dan folder berisi gambar.")
            sys.exit(1)
        
        # Split data
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
        )
        
        print(f"\n{'='*60}")
        print("DATA SPLIT (SEBELUM AUGMENTASI)")
        print("="*60)
        print(f"  Train: {len(X_train)}")
        print(f"  Validation: {len(X_val)}")
        print(f"  Test: {len(X_test)}")
        
        # Augmentasi
        if USE_AUGMENTATION:
            X_train_aug, y_train_aug = augment_data(X_train, y_train, augmentation_factor=AUGMENTATION_FACTOR)
            print(f"\nData training setelah augmentasi: {len(X_train_aug)}")
        else:
            X_train_aug, y_train_aug = X_train, y_train
        
        # Train model
        print(f"\n{'='*60}")
        print("TRAINING MODEL")
        print("="*60)
        
        model = train_model(X_train_aug, y_train_aug, X_val, y_val, fast_mode=FAST_MODE)
        
        # Evaluate
        print(f"\n{'='*60}")
        print("EVALUATION ON TEST SET")
        print("="*60)
        evaluate_model(model, X_test, y_test)
        
        # Analisis kesalahan
        analyze_misclassification(model, X_test, y_test)
        
        # Save model
        save_model(model, MODEL_PATH)
        
        overall_elapsed = time.time() - overall_start
        print(f"\n{'='*60}")
        print(f"⏱️  TOTAL WAKTU: {overall_elapsed:.1f} detik ({overall_elapsed/60:.1f} menit)")
        print("="*60)
    
    # ==========================================
    # 2. LAUNCH GRADIO
    # ==========================================
    print(f"\n{'='*60}")
    print("LAUNCHING GRADIO INTERFACE")
    print("="*60)
    print("\n🚀 Starting Gradio interface...")
    print("📱 Open the URL in your browser to test the model")
    print("\n💡 Tips:")
    print("   - Set USE_SAVED_MODEL = True untuk skip training di run berikutnya")
    print("   - Set FAST_MODE = True untuk training cepat (~2 menit)")
    print("   - Set FAST_MODE = False untuk akurasi maksimal (~8 menit)")
    print("\n")
    
    demo = create_gradio_interface(model)
    demo.launch(
        share=False,  # Set to True if you want public link
        server_name="127.0.0.1",
        server_port=6971
    )